In [25]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

In [37]:
from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
quotes=['USD','BTC']
sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1515283200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1522540801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [38]:
R.shape

(85, 90)

In [28]:
factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for sym in sysm:
    sym_row=[]
    for name in factor_names1:
        with open(path+name+sym+finpath1) as file:
            x=json.load(file)
        sym_row.append(x)
    for name in factor_names2:
        with open(path+name+sym+finpath2) as file:
            x=json.load(file)
        sym_row.append(x)
    for name in factor_names3:
        symll=sym.split('_')
        with open(path+name+symll[0]+finpath3) as file:
            x=json.load(file)
        sym_row.append(x)
    sym_row.append(1)
    factors.append(sym_row)
    #print(sym)
factors=np.array(factors)
factors.shape    

(90, 7)

In [39]:
beta=np.linalg.lstsq(factors, R.T,rcond=None)
beta

(array([[ 3.25708664e-02,  7.19247081e-01, -4.85795177e-01,
          1.28518225e+00,  1.83050972e-01, -8.30516342e-02,
          1.88748578e-02,  5.46286696e-03, -9.64689540e-01,
          9.41554380e-01, -1.25496169e-01, -2.06134611e-01,
          3.75435472e-01,  8.20776825e-02, -2.15202267e-03,
          4.53653777e-02, -2.70317699e-01, -1.07662187e+00,
          9.47949267e-01,  1.68427681e-01, -2.15673482e+00,
          7.85818411e-01, -1.65745959e-01,  1.68013825e+00,
          1.76558227e-01, -1.59545477e-01,  5.63788334e-02,
         -1.74816025e-01, -2.17217871e-01, -4.88887458e-01,
          6.82503430e-02,  7.59648585e-02,  2.31389301e-01,
         -8.87362648e-02, -1.38738121e-01,  6.57234822e-02,
         -1.85189747e-02,  3.34293924e-02, -3.66105812e-01,
         -4.12725431e-01,  4.61895689e-01, -1.10939673e-01,
          2.39676359e-02,  4.48625345e-01, -3.01811053e-01,
         -2.80149488e-02, -9.64568668e-02,  5.37323578e-02,
         -2.34630046e-02, -1.34216808e-0

In [103]:
gamma=beta[0]

In [177]:
print(R.T[0], (R.T-np.dot(factors, gamma))[0] )

[-0.05392268 -0.08252385 -0.03462123  0.02956491 -0.11276887  0.03911782
  0.02797964 -0.04267057 -0.00110561 -0.18684571 -0.0110527  -0.00098212
  0.03092923  0.10281155 -0.10166852 -0.06632892  0.00314323  0.0505971
 -0.02107934 -0.00588691  0.03045723  0.0240645  -0.04511775 -0.10463746
  0.01088072 -0.11274629 -0.0458793   0.05913283 -0.11901604 -0.16888199
  0.10553447 -0.01370841  0.08278507  0.10852938 -0.05586986 -0.07267666
  0.11366883 -0.05995378  0.10514564  0.05593775  0.01548176  0.08600883
 -0.06316068  0.07039029  0.0057201  -0.07111709 -0.06167311  0.03156552
 -0.04717504 -0.00925348  0.0730142   0.02409114 -0.02419493  0.05597597
  0.01056391  0.03720835  0.00377232 -0.00733648 -0.06225394 -0.07805594
 -0.06327043 -0.00734362 -0.04995657  0.08173656 -0.04345618  0.00166649
 -0.09736947 -0.00467406  0.00040069 -0.05082346  0.04306958  0.04791118
  0.0351429  -0.00134968 -0.02133837  0.02426655 -0.04381835 -0.00921045
 -0.03767158 -0.04370236  0.01870758 -0.11317197 -0.

In [142]:
cov=np.cov(R.T)

In [173]:
print(R.shape, gamma[0].shape, factors.shape)
#plot: given time, all returns vs their approx.

(85, 90) (85,) (90, 7)
